In [1]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import re
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse

In [2]:
df=pd.DataFrame()
list_product_name=[]
list_attribute=[]
list_value = []
list_cate_name=[]

In [5]:
url = 'https://medlineplus.gov/healthtopics.html'
soup_main=BeautifulSoup(requests.get(url).text,'html.parser')
bodyMaps_href=soup_main.find_all('li',type='bodymaps')
df = pd.DataFrame()
url_bodymaps = []
text_bodymaps = []
body_level_1 = []
body_level_2 = []
diseases_name = []
diseases_synonyms = []
diseases_link = []
symptoms_link = []
symptoms_name = []
source = []
for item in bodyMaps_href:
    anchor_tag = item.find('a')
    if anchor_tag:
        href = anchor_tag.get('href')
        text = anchor_tag.get_text()
        url_bodymaps.append(href)
        text_bodymaps.append(text)
for i in range(len(url_bodymaps)):
    body_lv1=BeautifulSoup(requests.get(url_bodymaps[i]).text,'html.parser')
    body_lv2=body_lv1.find('div',class_='tp_rdbox_bborder_c')
    body_lv3=body_lv2.find_all('ul')
    for ul in body_lv3:
        for anchor_tag in ul.find_all('a'):
            id_value = anchor_tag.get('id')
            text_value = anchor_tag.text
            id_value = id_value.replace("_menu", "")
            body_lv3_div = body_lv1.find_all('div',id=id_value)
            for div_element in body_lv3_div:
                li_elements = div_element.find_all('li')
                for li in li_elements:
                    link = li.find('a')
                    href = link.get('href')
                    text = link.get_text()
                    diseases = BeautifulSoup(requests.get(href).text,'html.parser')
                    alsocalled = diseases.find('span',class_='alsocalled')
                    if alsocalled is not None:
                        alsocalled_text = alsocalled.get_text()
                        alsocalled_text = alsocalled_text.replace("Also called: ", "")
                    else:
                        alsocalled_text = ''
                    pattern = re.compile(r'(?=.*symptoms)(?:(?=.*include)|(?=.*:))', re.IGNORECASE)
                    p_and_h3_tags_with_two_of_three = diseases.find_all(lambda tag: (tag.name == 'p' or tag.name == 'h3') and pattern.search(tag.text))
                    if len(p_and_h3_tags_with_two_of_three) != 0:
                        print(href)
                        for tag in p_and_h3_tags_with_two_of_three:
                            ul_tag = tag.find_next('ul')
                            if ul_tag and ul_tag.find_previous_sibling() == tag:
                                symptoms = []
                                li_tags = ul_tag.find_all('li')
                                for li_tag in li_tags:
                                    strong_tag = li_tag.find('strong')
                                    if strong_tag:
                                        symptom = strong_tag.text
                                    else:
                                        symptom = li_tag.text
                                    symptoms.append(symptom)
                                for symptom in symptoms:
                                    body_level_1.append(text_bodymaps[i])
                                    body_level_2.append(text_value)
                                    diseases_name.append(text)
                                    diseases_synonyms.append(alsocalled_text)
                                    diseases_link.append(href)
                                    symptoms_link.append(href)
                                    symptoms_name.append(symptom)
                                    source.append('medlineplus.gov')
                    else:
                        diseases_art = diseases.find_all('div',id='section51')
                        for div_dis in diseases_art:
                            li_dis = div_dis.find_all('li')
                            for li_ in li_dis:
                                link_dis = li_.find('a')
                                href_dis = link_dis.get('href')
                                soure_parse = urlparse(href_dis)
                                domain = soure_parse.netloc
                                if domain == 'www.nhlbi.nih.gov':
                                    nhlbi_symptoms = BeautifulSoup(requests.get(href_dis).text, 'html.parser')
                                    pattern = re.compile(r'(?=.*symptoms)(?:(?=.*include)|(?=.*:))', re.IGNORECASE)
                                    p_and_h3_tags_with_two_of_three = nhlbi_symptoms.find_all(lambda tag: (tag.name == 'p' or tag.name == 'h3') and pattern.search(tag.text))
                                    if len(p_and_h3_tags_with_two_of_three) != 0:
                                        print(href_dis)
                                        for tag in p_and_h3_tags_with_two_of_three:
                                            ul_tag = tag.find_next('ul')
                                            if ul_tag and ul_tag.find_previous_sibling() == tag:
                                                symptoms = []
                                                li_tags = ul_tag.find_all('li')
                                                for li_tag in li_tags:
                                                    strong_tag = li_tag.find('strong')
                                                    if strong_tag:
                                                        symptom = strong_tag.text
                                                    else:
                                                        symptom = li_tag.text
                                                    symptoms.append(symptom)
                                                for symptom in symptoms:
                                                    body_level_1.append(text_bodymaps[i])
                                                    body_level_2.append(text_value)
                                                    diseases_name.append(text)
                                                    diseases_synonyms.append(alsocalled_text)
                                                    diseases_link.append(href)
                                                    symptoms_link.append(href_dis)
                                                    symptoms_name.append(symptom)
                                                    source.append(domain)
                                    else:
                                        symptoms = href_dis + '/symptoms'
                                        print(symptoms)
                                        symptoms_get = BeautifulSoup(requests.get(symptoms).text,'html.parser')
                                        keyword = "clearfix text-formatted field field--name-field-component-section-"
                                        symptoms_details = symptoms_get.find_all('div', class_=lambda label: label and keyword.lower() in label.lower())
                                        for div_symtom in symptoms_details:
                                            li_symtom = div_symtom.find_all('li')
                                            for li_sym in li_symtom:
                                                try:
                                                    text_syms = li_sym.find('strong')
                                                    text_sym = text_syms.get_text()
                                                    parsed_url = urlparse(href_dis)
                                                    domain = parsed_url.netloc
                                                    body_level_1.append(text_bodymaps[i])
                                                    body_level_2.append(text_value)
                                                    diseases_name.append(text)
                                                    diseases_synonyms.append(alsocalled_text)
                                                    diseases_link.append(href)
                                                    symptoms_link.append(symptoms)
                                                    symptoms_name.append(text_sym)
                                                    source.append(domain)
                                                except:
                                                    text_sym = li_sym.get_text()
                                                    parsed_url = urlparse(href_dis)
                                                    domain = parsed_url.netloc
                                                    body_level_1.append(text_bodymaps[i])
                                                    body_level_2.append(text_value)
                                                    diseases_name.append(text)
                                                    diseases_synonyms.append(alsocalled_text)
                                                    diseases_link.append(href)
                                                    symptoms_link.append(symptoms)
                                                    symptoms_name.append(text_sym)
                                                    source.append(domain)
                                elif domain == 'www.mayoclinic.org':
                                    print(href_dis)
                                    mayo_symptoms = BeautifulSoup(requests.get(href_dis).text, 'html.parser')
                                    pattern = re.compile(r'(?=.*symptoms)(?:(?=.*include)|(?=.*:))', re.IGNORECASE)
                                    p_and_h3_tags_with_two_of_three = mayo_symptoms.find_all(lambda tag: (tag.name == 'p' or tag.name == 'h3') and pattern.search(tag.text))
                                    for tag in p_and_h3_tags_with_two_of_three:
                                        ul_tag = tag.find_next('ul')
                                        if ul_tag and ul_tag.find_previous_sibling() == tag:
                                            symptoms = []
                                            li_tags = ul_tag.find_all('li')
                                            for li_tag in li_tags:
                                                strong_tag = li_tag.find('strong')
                                                if strong_tag:
                                                    symptom = strong_tag.text
                                                else:
                                                    symptom = li_tag.text
                                                symptoms.append(symptom)
                                            for symptom in symptoms:
                                                body_level_1.append(text_bodymaps[i])
                                                body_level_2.append(text_value)
                                                diseases_name.append(text)
                                                diseases_synonyms.append(alsocalled_text)
                                                diseases_link.append(href)
                                                symptoms_link.append(href_dis)
                                                symptoms_name.append(symptom)
                                                source.append(domain)
                                elif domain == 'www.niddk.nih.gov':
                                    mayo_symptoms = BeautifulSoup(requests.get(href_dis).text, 'html.parser')
                                    pattern = re.compile(r'(?=.*symptoms)(?:(?=.*include)|(?=.*:))', re.IGNORECASE)
                                    parts_url = url.split("/")
                                    result_url = "/".join(parts_url[:6])
                                    p_and_h3_tags_with_two_of_three = mayo_symptoms.find_all(lambda tag: (tag.name == 'p' or tag.name == 'h3') and pattern.search(tag.text))
                                    if len(p_and_h3_tags_with_two_of_three) != 0 and result_url != href_dis:
                                        for tag in p_and_h3_tags_with_two_of_three:
                                            ul_tag = tag.find_next('ul')
                                            if ul_tag and ul_tag.find_previous_sibling() == tag:
                                                symptoms = []
                                                li_tags = ul_tag.find_all('li')
                                                for li_tag in li_tags:
                                                    strong_tag = li_tag.find('strong')
                                                    if strong_tag:
                                                        symptom = strong_tag.text
                                                    else:
                                                        symptom = li_tag.text
                                                    symptoms.append(symptom)
                                                for symptom in symptoms:
                                                    body_level_1.append(text_bodymaps[i])
                                                    body_level_2.append(text_value)
                                                    diseases_name.append(text)
                                                    diseases_synonyms.append(alsocalled_text)
                                                    diseases_link.append(href)
                                                    symptoms_link.append(href_dis)
                                                    symptoms_name.append(symptom)
                                                    source.append(domain)
                                    else:
                                        final_url = result_url + '/symptoms-causes'
                                        symptoms = BeautifulSoup(requests.get(final_url).text, 'html.parser')
                                        pattern = re.compile(r'(?=.*symptoms)(?:(?=.*include)|(?=.*:))', re.IGNORECASE)
                                        p_and_h3_tags_with_two_of_three = symptoms.find_all(lambda tag: (tag.name == 'p' or tag.name == 'h3') and pattern.search(tag.text))
                                        for tag in p_and_h3_tags_with_two_of_three:
                                            ul_tag = tag.find_next('ul')
                                            if ul_tag and ul_tag.find_previous_sibling() == tag:
                                                symptoms = []
                                                li_tags = ul_tag.find_all('li')
                                                for li_tag in li_tags:
                                                    strong_tag = li_tag.find('strong')
                                                    if strong_tag:
                                                        symptom = strong_tag.text
                                                    else:
                                                        symptom = li_tag.text
                                                    symptoms.append(symptom)
                                                for symptom in symptoms:
                                                    body_level_1.append(text_bodymaps[i])
                                                    body_level_2.append(text_value)
                                                    diseases_name.append(text)
                                                    diseases_synonyms.append(alsocalled_text)
                                                    diseases_link.append(href)
                                                    symptoms_link.append(final_url)
                                                    symptoms_name.append(symptom)
                                                    source.append(domain)
                                else:
                                    #print('else')
                                    text_sym = ''
                                    body_level_1.append(text_bodymaps[i])
                                    body_level_2.append(text_value)
                                    diseases_name.append(text)
                                    diseases_synonyms.append(alsocalled_text)
                                    diseases_link.append(href)
                                    symptoms_link.append(href_dis)
                                    symptoms_name.append(text_sym)
                                    source.append(domain)
                    #break
    #         break
    #     break
    # break
df['body_level_1'] = body_level_1
df['body_level_2'] = body_level_2
df['diseases_name'] = diseases_name
df['diseases_synonyms'] = diseases_synonyms
df['diseases_link'] = diseases_link
df['symptoms_link'] = symptoms_link
df['symptoms_name'] = symptoms_name
df['source'] = source
df['diseases_synonyms'] = df['diseases_synonyms'].str.split(', ')
df = df.explode('diseases_synonyms', ignore_index=True)
df

https://www.nhlbi.nih.gov/health/aortic-aneurysm/symptoms
https://www.nhlbi.nih.gov/health/aortic-aneurysm/symptoms
https://medlineplus.gov/arteriovenousmalformations.html
https://www.nhlbi.nih.gov/health/atherosclerosis/symptoms
https://medlineplus.gov/bloodclots.html
https://medlineplus.gov/brainaneurysm.html
https://www.mayoclinic.org/diseases-conditions/carotid-artery-disease/symptoms-causes/syc-20360519?p=1
https://medlineplus.gov/giantcellarteritis.html
https://www.nhlbi.nih.gov/health/high-blood-pressure/symptoms
https://medlineplus.gov/kawasakidisease.html
https://www.mayoclinic.org/diseases-conditions/low-blood-pressure/symptoms-causes/syc-20355465?p=1
https://www.nhlbi.nih.gov/health/low-blood-pressure
https://medlineplus.gov/peripheralarterialdisease.html
https://www.mayoclinic.org/diseases-conditions/raynauds-disease/symptoms-causes/syc-20363571?p=1
https://medlineplus.gov/shock.html
https://medlineplus.gov/stroke.html
https://medlineplus.gov/transientischemicattack.html
ht

,body_level_1,body_level_2,diseases_name,diseases_synonyms,diseases_link,symptoms_link,symptoms_name,source
0,"Blood, Heart and Circulation",Arteries,Aortic Aneurysm,,https://medlineplus.gov/aorticaneurysm.html,https://www.nhlbi.nih.gov/health/aortic-aneury...,Difficult or painful swallowing,www.nhlbi.nih.gov
1,"Blood, Heart and Circulation",Arteries,Aortic Aneurysm,,https://medlineplus.gov/aorticaneurysm.html,https://www.nhlbi.nih.gov/health/aortic-aneury...,Difficulty breathing,www.nhlbi.nih.gov
2,"Blood, Heart and Circulation",Arteries,Aortic Aneurysm,,https://medlineplus.gov/aorticaneurysm.html,https://www.nhlbi.nih.gov/health/aortic-aneury...,Feeling full,www.nhlbi.nih.gov
3,"Blood, Heart and Circulation",Arteries,Aortic Aneurysm,,https://medlineplus.gov/aorticaneurysm.html,https://www.nhlbi.nih.gov/health/aortic-aneury...,Hoarseness,www.nhlbi.nih.gov
4,"Blood, Heart and Circulation",Arteries,Aortic Aneurysm,,https://medlineplus.gov/aorticaneurysm.html,https://www.nhlbi.nih.gov/health/aortic-aneury...,Pain,www.nhlbi.nih.gov
...,...,...,...,...,...,...,...,...
4538,Male Reproductive System,Testicle,Testicular Cancer,,https://medlineplus.gov/testicularcancer.html,https://medlineplus.gov/testicularcancer.html,A change in how the testicle feels,medlineplus.gov
4539,Male Reproductive System,Testicle,Testicular Cancer,,https://medlineplus.gov/testicularcancer.html,https://medlineplus.gov/testicularcancer.html,A dull ache in the lower abdomen (belly) or th...,medlineplus.gov
4540,Male Reproductive System,Testicle,Testicular Cancer,,https://medlineplus.gov/testicularcancer.html,https://medlineplus.gov/testicularcancer.html,A sudden build-up of fluid in the scrotum,medlineplus.gov
4541,Male Reproductive System,Testicle,Testicular Cancer,,https://medlineplus.gov/testicularcancer.html,https://medlineplus.gov/testicularcancer.html,Pain or discomfort in a testicle or in the scr...,medlineplus.gov


In [6]:
df.to_excel('D:/MedlinePlus_test_2.xlsx', index=False)

In [7]:
href = 'https://medlineplus.gov/ency/article/003122.htm'
diseases = BeautifulSoup(requests.get(href).text,'html.parser')
pattern = re.compile(r'(?=.*symptoms)(?:(?=.*include)|(?=.*:))', re.IGNORECASE)
p_and_h3_tags_with_two_of_three = diseases.find_all(lambda tag: (tag.name == 'p' or tag.name == 'h3') and pattern.search(tag.text))
for tag in p_and_h3_tags_with_two_of_three:
    ul_tag = tag.find_next('ul')
    if ul_tag and ul_tag.find_previous_sibling() == tag:
        symptoms = []
        li_tags = ul_tag.find_all('li')
        for li_tag in li_tags:
            strong_tag = li_tag.find('strong')
            if strong_tag:
                symptom = strong_tag.text
            else:
                symptom = li_tag.text
            symptoms.append(symptom)
        for symptom in symptoms:
            print(symptom)

Absent menstrual period
Diarrhea
Excessive fatigue
Excessive gas or belching
Irritability
Vomiting
Weight gain or loss
